<a href="https://colab.research.google.com/github/djs21905/Machine-Learning/blob/master/SmithAssignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ingest






In [0]:
import zipfile
import requests
from io import BytesIO
import numpy as np
from PIL import Image
from tqdm import tqdm 


# Data Extraction/Conversion to ZIP --> numpy array
def ziptonumpy(url,train = False):
  size = (200,200)
  training_data = []
  labels = []
  try:
    request = requests.get(url)
    zipped = zipfile.ZipFile(BytesIO(request.content))
    inflist = zipped.infolist()[1:]
    if train == False: 
      for img in tqdm(inflist):
        content = Image.open(zipped.open(img))
        img = content.resize(size)
        training_data.append(np.array(img))
    elif train == True: 
      for img in tqdm(inflist):
        label = img.filename[6:]
        label_int = 1
        content = Image.open(zipped.open(img))
        img = content.resize(size)
        if "cat" in label:
          label_int = 0
        else:
          pass
        training_data.append((np.array(img),label_int))
  except:
    print("An error was encountered")
  return np.array(training_data)

In [0]:
train_url = "https://msds422smith.s3.amazonaws.com/train.zip"
test_url = "https://msds422smith.s3.amazonaws.com/test.zip"

test_data = ziptonumpy(url = test_url)
train_data = ziptonumpy(url = train_url, train = True)

100%|██████████| 25000/25000 [01:11<00:00, 347.42it/s]


In [0]:
test_data = test_data/255

In [0]:
X,y = np.split(train_data, [-1], axis=1)

X = np.array([x[0] for x in X])
X = X/255

In [0]:
X.shape , type(test_data)

((25000, 200, 200, 3), numpy.ndarray)

# Model

In [0]:
np.random.seed(1337)

from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPool2D
from keras.callbacks import History 

history = History()
model = Sequential()

model.add(Conv2D(34, (3,3), input_shape=X[0].shape, activation='relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, (1,1), activation='relu'))


model.add(Flatten())
model.add(Dropout(0.4))

model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"])

Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
output = model.fit(X,y,batch_size=32, epochs=10, validation_split=0.1,verbose = True, callbacks=[history])




Train on 22500 samples, validate on 2500 samples
Epoch 1/10





22500/22500 [==============================] - 348s 15ms/step - loss: 0.6881 - acc: 0.5546 - val_loss: 0.7785 - val_acc: 0.0000e+00
Epoch 2/10
22500/22500 [==============================] - 356s 16ms/step - loss: 0.6790 - acc: 0.5619 - val_loss: 0.7972 - val_acc: 0.0064
Epoch 3/10
22500/22500 [==============================] - 349s 16ms/step - loss: 0.6347 - acc: 0.6361 - val_loss: 0.6367 - val_acc: 0.6092
Epoch 4/10
22500/22500 [==============================] - 344s 15ms/step - loss: 0.5471 - acc: 0.7281 - val_loss: 0.9839 - val_acc: 0.4448
Epoch 5/10
22500/22500 [==============================] - 344s 15ms/step - loss: 0.4690 - acc: 0.7787 - val_loss: 0.4033 - val_acc: 0.8248
Epoch 6/10
22500/22500 [==============================] - 343s 15ms/step - loss: 0.4178 - acc: 0.8095 - val_loss: 0.7228 - val_acc: 0.6348
Epoch 7/10
22500/22500 [==============================] - 344s 15ms/step - loss: 0.3804 - acc: 0.8336 - v

In [0]:
#model serialization workaround

import pickle 

pickle.dump(output , open("model_4.sav","wb"))
history_data = dict(output.history)
pickle.dump(history_data , open("model_4_hist.sav","wb"))

In [0]:
#verifying my serialization / logic worked

test =pickle.load(open("model_4.sav",'rb'))
test2 = pickle.load(open("model_4_hist.sav",'rb'))



Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




# Model Evalutation

In [0]:
import pickle
from urllib.request import urlopen

model_1_history = "https://msds422smith.s3.amazonaws.com/assignment7models/model_1_hist.sav"
model_1 = "https://msds422smith.s3.amazonaws.com/assignment7models/model_1.sav"
model_2_history = "https://msds422smith.s3.amazonaws.com/assignment7models/model_2_hist.sav"
model_2 = "https://msds422smith.s3.amazonaws.com/assignment7models/model_2.sav"
#model_3_history = 
#model_3 = 

#baseline_model = pickle.load(urlopen("https://msds422smith.s3.amazonaws.com/baseline_model.sav"))

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPool2D
from keras.callbacks import History 

models = [model_1, model_2]
names = ["model_1_pred","model_2_pred"]
results = []

for model, name in zip(models,names):
  name = pickle.load(urlopen(model))
  a = name.predict_proba(test_data)
  results.append(a)

#baseline_results = baseline_model.predict_proba(test_data)
#baseline_results



# Export Test Results to Kaggle

In [0]:
import pandas as pd

#shows probability of class 1 = dog
kaggle_export = pd.DataFrame(results[1], index = np.arange(1,12501,1) , columns = ["label"])
kaggle_export.index.name = 'id'
kaggle_export.to_csv("model_2.csv")
kaggle_export

,label
id,
1,0.789493
2,0.002917
3,0.141585
4,0.999129
5,0.754810
...,...
12496,0.249166
12497,0.942540
12498,0.693904


In [0]:
history_1 = pickle.load(urlopen(model_1_history))
history_2 = pickle.load(urlopen(model_2_history))

In [0]:
train_acc_1 = history_1['acc'][9]
val_acc_1 = history_1['val_acc'][9]

train_acc_2 = history_2['acc'][9]
val_acc_2 = history_2['val_acc'][9]

final_comparison = pd.DataFrame({"Train_acc":[train_acc_1,train_acc_2], "Val_acc":[val_acc_1,val_acc_2], "log_loss_score":[1.81967, 1.86263]}, index = ["model1","model2"])



In [0]:
final_comparison

,Train_acc,Val_acc,log_loss_score
model1,0.879289,0.8648,1.81967
model2,0.879733,0.7428,1.86263
